## Site Access Score Tool

### May 26th, 2022
<br>

--------------------------------------------------------------------------------------------


#### Round 1

* GetParameterAsText (https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/getparameterastext.htm)
* SpatialReference (https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/spatialreference.htm)
* PointGeometry (https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/pointgeometry.htm)
* Copy Features (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/copy-features.htm)
* Buffer Analysis (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/buffer.htm)
* Intersect Analysis (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/intersect.htm)
* os.path.join() (https://www.geeksforgeeks.org/python-os-path-join-method/)

1) Created a point with longitude, latitude number (double) <br>
2) Created a buffer with a customized distance (linear unit) <br> 
3) Created point feature layers intersected between the buffer and major roads <br> 


--------------------------------------------------------------------------------------------


#### Round 2

* OriginDestinationCostMatrix (https://pro.arcgis.com/en/pro-app/2.8/arcpy/network-analyst/odcostmatrix.htm)
* Join Field (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/join-field.htm)
* Copy (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/copy.htm)
* Table to Excel (Conversion) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/conversion/table-to-excel.htm)
* Summary Statistics (Analysis) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/summary-statistics.htm)
* UpdateCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/updatecursor-class.htm)
  * SearchCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/searchcursor-class.htm)
  * InsertCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/insertcursor-class.htm)
* AddMessage (https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/addmessage.htm)
* Delete (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/delete.htm)

1) Added workspace as a parameter for different environment settings <br>
2) Created drive times(minutes) and network distance(miles) from buffered points intersected with major roads to a target point <br>
3) Used our Network Dataset, not esri's one <br>
4) Exported excel(xlsx) files for the OD Cost Matrix as well as the summary calculation. <br>
5) Added each point's drive time and network distance to the result message. <br>


--------------------------------------------------------------------------------------------


#### Next Step - Round 3 (by June 13)

1) Multiple Input Locations & Buffer Distance with csv file
>| Tips |
<br>
* Use csv for the location inputs (e.g. Fields = ["ID", "Location", "Lat", "Long", "Buffer_Distance"]) <br>
* Need to change input parameter as csv file and use "for loop" to use all rows as inputs <br>
>

2) Calculate the number of turns
>| Tips |
<br>
* Copy Traversed Source Features (https://desktop.arcgis.com/en/arcmap/latest/tools/network-analyst-toolbox/copy-traversed-source-features.htm)
>


--------------------------------------------------------------------------------------------

#### Round 3 - 220531

1. One Site to Multiple Sites <br>
  * Change the in_lat, in_long, and buffer_dist parameters to the csv parameters with the columns <br>
    * 1) Use XYTableToPoint Tool for in_lat and in_long <br>
    * 2) Create GetParameterAsText for Buffer Distance <br>
<br>

>| Reference |
<br>
* Iterating Output Names on Feature Class (https://gis.stackexchange.com/questions/206401/making-lists-looping-a-function-through-a-list-and-controlling-output-names)
>


In [ ]:
### 220531 Site Access Score | Round 3

# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

###################################################################################################
# 1. One Site to Multiple Sites

# Change the in_lat, in_long, and buffer_dist parameters to the csv parameters with the columns
# 1) Use XYTableToPoint Tool for in_lat and in_long
# 2) Set the distance column to a new parameter and use it instead of buffer_dist in Buffer_Analysis tool

###################################################################################################

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
location_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
# in_lat = arcpy.GetParameterAsText(1) # Double, Y
# in_long = arcpy.GetParameterAsText(2) # Double, X
point_target = arcpy.GetParameterAsText(2) # feature layer - Target Point
# point_buffer = arcpy.GetParameterAsText(4) # feature layer - Buffer Area
buffer_dist = arcpy.GetParameterAsText(3) # Linear unit - Buffer Distance
output_point = arcpy.GetParameterAsText(4) # feature layer
output_point_xlsx = arcpy.GetParameterAsText(5) # xlsx
output_table_xlsx = arcpy.GetParameterAsText(6) # xlsx

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

# Create point features for the target locations
xy_point = os.path.join(workspace, "point")
arcpy.management.XYTableToPoint(
    in_table = location_csv, 
    out_feature_class = xy_point,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_target, point_buffer, buffer_dist)

# create intersections between the buffer and major roads feature layer
  # Set local variable
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "intsct_point")
arcpy.Intersect_analysis([point_buffer, major_roads], intsct_point, "", "", "point")

# set local variables to perform origin destination cost matrix
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origin = intsct_point
input_destination = point_target

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultDestinationCount = 1
odcm.distanceUnits = arcpy.nax.DistanceUnits.Miles
odcm.lineShapeType = arcpy.nax.LineShapeType.StraightLine
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origin)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destination)
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
output_lines = os.path.join(workspace, "output_lines")
result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)

# Table join with the point feuatres 
field = "OBJECTID"
arcpy.JoinField_management(intsct_point, field, 
                           output_lines, field)
arcpy.Copy_management(intsct_point, output_point)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_point, output_point_xlsx)

# Execute Summary Statistics and create a table
output_table = os.path.join(workspace, "output_table")
arcpy.Statistics_analysis(
    in_table = output_lines, 
    out_table = output_table, 
    statistics_fields = [["Total_Time", "MEAN"],
                         ["Total_Time", "MAX"],
                         ["Total_Time", "MIN"],
                         ["Total_Time", "MEDIAN"],
                         ["Total_Time", "RANGE"],
                         ["Total_Distance", "MEAN"],
                         ["Total_Distance", "MAX"],
                         ["Total_Distance", "MIN"],
                         ["Total_Distance", "MEDIAN"],
                         ["Total_Distance", "RANGE"],
                        ])

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_table, output_table_xlsx)

# Use ORDER BY sql clause to sort field values
fields = ['OID@', 'Total_Time', 'Total_Distance']
with arcpy.da.UpdateCursor(
    in_table = output_lines, 
    field_names = fields, 
    sql_clause=(None, 'ORDER BY Total_Time DESC')) as cursor:
    for row in cursor:
        row[1] = round(float(row[1]), 1)
        row[2] = round(float(row[2]), 2)
        cursor.updateRow(row)
        arcpy.AddMessage('ID: {}, Drive Time: {} Minutes, Drive Distance: {} Miles'.format(row[0], row[1], row[2]))
    
# Delete unnecessary data
arcpy.management.Delete(intsct_point)
arcpy.management.Delete(output_lines)
arcpy.management.Delete(output_table)


In [ ]:
# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

# For each row, print the fields of feature class
fields = ['ID', 'Lat', 'Long', 'Buffer_Distance']
sr = arcpy.SpatialReference("WGS 1984")
with arcpy.da.SearchCursor(loc_csv, fields) as cursor:
    for row in cursor:
        pointNum = 'point_{}'.format(row[0])
        pointGeom_Num = 'pointGeom_{}'.format(row[0])
        pointFlyr_Num = 'pointFlyr_{}'.format(row[0])
        pointBuff_Num = 'pointBuff_{}'.format(row[0])
        pointBuff_Dis = '{} Miles'.format(row[3])
        pointNum = arcpy.Point(row[2],row[1])
        pointGeom_Num = arcpy.PointGeometry(pointNum,sr)
        arcpy.CopyFeatures_management(pointGeom_Num, pointFlyr_Num)
        arcpy.Buffer_analysis(pointFlyr_Num, pointBuff_Num, pointBuff_Dis)

###################################################################################################

# Reference for 220601

# Separate the For Loop as 2 steps
# 1) Create Points and Merge them as one feature class
# 2) Create Buffer Features and Merge them as one feature class


In [ ]:
# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
loc_csv = arcpy.GetParameterAsText(1) # employee csv, Table or Table view
xy_point = arcpy.GetParameterAsText(2) # Feature Layer

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

### 1-2. Current Version
# Create point features
arcpy.management.XYTableToPoint(
    in_table = loc_csv, 
    out_feature_class = xy_point,
    x_field = "Long", 
    y_field = "Lat",
    coordinate_system = arcpy.SpatialReference("WGS 1984"))


In [ ]:
# Reference 1

mxd = arcpy.mapping.MapDocument("Current")         
df = arcpy.mapping.ListDataFrames(mxd)[0]         
fc = 'Database Connections/TDPMode.sde/KWAGISMAIN.DBO.Parcels'         
field = "OBJECTID"         
field1 = "LGA"         
field2 = "District"         
field3 = "Block_No"         
field4 = "Plot_No"         
rows = arcpy.SearchCursor(fc)         
row = rows.next()         
while row:             
    val = row.getValue(field)             
    val1 = row.getValue(field1)             
    val2 = row.getValue(field2)             
    val3 = row.getValue(field3)             
    val4 = row.getValue(field4)             
    whereClause = '"OBJECTID"' + " = '" + str(val) + "'"             
    outName = "Block_" + str(val3) + "_Plot_" + str(val4) + "_" + str(val2) + "_Area_of_" + str(val1) + "_LGA" + ".pdf"             
    path = "C:/Users/Administrator/Documents/ArcGIS/Default.gdb/"      
    
arcpy.SelectLayerByAttribute_management("Parcels", "NEW_SELECTION", whereClause)             

arcpy.Buffer_analysis ("Parcels", "DataDrivenPage_Buffer", "14 meters", "FULL", "ROUND", "NONE")             

layer_list = "Lines","Parcels","Access_Road_Graphics","Points","Lines_Split"             

for layer in layer_list:                 
    arcpy.SelectLayerByLocation_management(layer, "COMPLETELY_WITHIN", "DataDrivenPage_Buffer", "", "NEW_SELECTION")             
    exportLayer = "Points", "Lines", "Parcels","Lines_Split","Access_Road_Graphics"             

for layer in exportLayer:                 
    outFC = path + layer + "_New"                  
    arcpy.Clip_analysis(layer,"DataDrivenPage_Buffer",outFC)                 

arcpy.SelectLayerByAttribute_management(layer, "CLEAR_SELECTION")             
mxd = arcpy.mapping.MapDocument("current")          
lyr = arcpy.mapping.ListLayers(mxd, "Lines_New")[0]             

for lblClass in lyr.labelClasses:                 
    lblClass.SQLQuery = '"ParcelID"' + "=" + str(val)                  
    arcpy.RefreshActiveView()             

arcpy.SelectLayerByAttribute_management("DataDrivenPage_Buffer", "NEW_SELECTION", "OBJECTID = 1")             
df.zoomToSelectedFeatures()             
mxd = arcpy.mapping.MapDocument("Current")             

for df in arcpy.mapping.ListDataFrames(mxd):                 
    df.rotation = 0             
    if df.scale <= 400:                  
        df.scale = 500             
        if df.scale > 400 and df.scale < 1000:                 
            df.scale = 1000                 
            else:                  
                df.scale = 2000             
                
arcpy.mapping.ExportToPDF(mxd,r"C:\STATE\\Batch\\" + "TDP_For_" + outName)             
row = rows.next()

In [ ]:
# Reference 2

print ('Comparing Pending against Confirmed...')
##Select ScotWind_Applications_Confirmed by location within 5km of the selected row of ScotWind_Applications_Pending##
with arcpy.da.SearchCursor("ScotWind_Applications_Pending", ["OBJECTID", "ScotWind_Applications_SMPJoin_AppID", "Rank", "SHAPE@AREA"], sql_clause=(None, "ORDER BY Rank DESC")) as cursor:
    for row in cursor:
        ##Select the current row in ScotWind_Applications_Pending##
        arcpy.SelectLayerByAttribute_management ("ScotWind_Applications_Pending","NEW_SELECTION", "OBJECTID = {}".format(row[0]))
        
        ##Create a Layer of all Confirmed Applications which is updated on each iteration##
        arcpy.MakeFeatureLayer_management (ScotWind_Applications_Scoring, "ScotWind_Applications_Confirmed")
        arcpy.management.SelectLayerByAttribute ("ScotWind_Applications_Confirmed", "NEW_SELECTION", "Status = 'Confirmed'", None)

        area_sum = 0
        with arcpy.da.SearchCursor("ScotWind_Applications_Confirmed", ["SHAPE@AREA"]) as cursor2:
           for row2 in cursor2:
                area_sum = area_sum + row2[0]
                                                     
        print (area_sum)

        universal_sum = area_sum + row[3]
        
        ##Select all Confirmed Applications on each iteration that are within 5km of the 
        arcpy.management.SelectLayerByLocation("ScotWind_Applications_Confirmed", "INTERSECT", "ScotWind_Applications_Pending", "5 Kilometers", "SUBSET_SELECTION", "NOT_INVERT")

        featurecount = int(arcpy.GetCount_management("ScotWind_Applications_Confirmed").getOutput(0))

        print (u'{0}, {1}, {2}, {3}'.format(row[0], row[1], row[2], row[3]))

        print (featurecount)

        print (universal_sum)
        
        codeblock5 = """def Reclass(Status):
            if universal_sum > 4500000000:
                return 'Capped Out'
            elif featurecount== 0:
                return 'Confirmed'
            else:
                return 'Defeated'"""
                
        arcpy.management.CalculateField("ScotWind_Applications_Pending", "Status", "Reclass(!Status!)", "PYTHON3", codeblock5)

        arcpy.Delete_management("ScotWind_Applications_Confirmed")‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍

In [ ]:
### 220526 Site Access Score Tool | 2nd Round

# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
in_lat = arcpy.GetParameterAsText(1) # Double, Y
in_long = arcpy.GetParameterAsText(2) # Double, X
point_target = arcpy.GetParameterAsText(3) # feature layer - Target Point
point_buffer = arcpy.GetParameterAsText(4) # feature layer - Buffer Area
buffer_dist = arcpy.GetParameterAsText(5) # Linear unit - Buffer Distance
output_point = arcpy.GetParameterAsText(6) # feature layer
output_point_xlsx = arcpy.GetParameterAsText(7) # xlsx
output_table_xlsx = arcpy.GetParameterAsText(8) # xlsx

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

# create a point with spatial reference
point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)
# create a point feature class
arcpy.CopyFeatures_management(point_geom, point_target)

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_target, point_buffer, buffer_dist)

# create intersections between the buffer and major roads feature layer
  # Set local variable
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "intsct_point")
arcpy.Intersect_analysis([point_buffer, major_roads], intsct_point, "", "", "point")

# set local variables to perform origin destination cost matrix
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origin = intsct_point
input_destination = point_target

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultDestinationCount = 1
odcm.distanceUnits = arcpy.nax.DistanceUnits.Miles
odcm.lineShapeType = arcpy.nax.LineShapeType.StraightLine
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origin)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destination)
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
output_lines = os.path.join(workspace, "output_lines")
result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)

# Table join with the point feuatres 
field = "OBJECTID"
arcpy.JoinField_management(intsct_point, field, 
                           output_lines, field)
arcpy.Copy_management(intsct_point, output_point)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_point, output_point_xlsx)

# Execute Summary Statistics and create a table
output_table = os.path.join(workspace, "output_table")
arcpy.Statistics_analysis(
    in_table = output_lines, 
    out_table = output_table, 
    statistics_fields = [["Total_Time", "MEAN"],
                         ["Total_Time", "MAX"],
                         ["Total_Time", "MIN"],
                         ["Total_Time", "MEDIAN"],
                         ["Total_Time", "RANGE"],
                         ["Total_Distance", "MEAN"],
                         ["Total_Distance", "MAX"],
                         ["Total_Distance", "MIN"],
                         ["Total_Distance", "MEDIAN"],
                         ["Total_Distance", "RANGE"],
                        ])

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_table, output_table_xlsx)

# Use ORDER BY sql clause to sort field values
fields = ['OID@', 'Total_Time', 'Total_Distance']
with arcpy.da.UpdateCursor(
    in_table = output_lines, 
    field_names = fields, 
    sql_clause=(None, 'ORDER BY Total_Time DESC')) as cursor:
    for row in cursor:
        row[1] = round(float(row[1]), 1)
        row[2] = round(float(row[2]), 2)
        cursor.updateRow(row)
        arcpy.AddMessage('ID: {}, Drive Time: {} Minutes, Drive Distance: {} Miles'.format(row[0], row[1], row[2]))
    
# Delete unnecessary data
arcpy.management.Delete(intsct_point)
arcpy.management.Delete(output_lines)
arcpy.management.Delete(output_table)
